In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session
/kaggle/input/2021-ai-quiz1-p4/train.csv
/kaggle/input/2021-ai-quiz1-p4/test.csv
/kaggle/input/2021-ai-quiz1-p4/submit_sample.csv
# 따릉이 예측문제
train = pd.read_csv("../input/2021-ai-quiz1-p4/train.csv")
test = pd.read_csv("../input/2021-ai-quiz1-p4/test.csv")
submit = pd.read_csv("../input/2021-ai-quiz1-p4/submit_sample.csv")
train.head()
id	hour	hour_bef_temperature	hour_bef_precipitation	hour_bef_windspeed	hour_bef_humidity	hour_bef_visibility	hour_bef_ozone	hour_bef_pm10	hour_bef_pm2.5	count
0	3	20	16.3	1.0	1.5	89.0	576.0	0.027	76.0	33.0	49.0
1	6	13	20.1	0.0	1.4	48.0	916.0	0.042	73.0	40.0	159.0
2	7	6	13.9	0.0	0.7	79.0	1382.0	0.033	32.0	19.0	26.0
3	8	23	8.1	0.0	2.7	54.0	946.0	0.040	75.0	64.0	57.0
4	9	18	29.5	0.0	4.8	7.0	2000.0	0.057	27.0	11.0	431.0
test.head()
id	hour	hour_bef_temperature	hour_bef_precipitation	hour_bef_windspeed	hour_bef_humidity	hour_bef_visibility	hour_bef_ozone	hour_bef_pm10	hour_bef_pm2.5
0	1655	4	14.6	0.0	0.6	49.0	2000.0	0.014	41.0	27.0
1	1657	20	21.8	0.0	4.4	40.0	2000.0	0.048	35.0	19.0
2	1660	22	18.3	0.0	1.0	37.0	970.0	0.032	169.0	38.0
3	1662	7	13.2	0.0	0.6	85.0	431.0	0.023	50.0	39.0
4	1663	8	8.9	0.0	0.9	77.0	613.0	0.005	82.0	40.0
submit.head()
id	count
0	0	0
1	1	0
2	2	0
3	3	0
4	4	0
#필요없는 column들을 제거하고 x_train, y_train, x_test 셋팅
x_train = train.drop(['id', 'count'], axis = 1)
y_train = train['count']
x_test = test.drop('id', axis = 1)
import torch
import torch.nn
import torch.optim as optim
import torch.nn.functional as F
import torch.cuda
# 정규화를 위한 sklearn 호출 후 정규화 진행
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
x_train = pd.DataFrame(sc.fit_transform(x_train))
x_test = pd.DataFrame(sc.transform(x_test))
#tensor로 변환
x_train = torch.FloatTensor(x_train.values).cuda()
y_train = torch.FloatTensor(y_train.values).cuda()
x_test = torch.FloatTensor(x_test.values).cuda()
# linear의 입력 노드 확인을 위한 shape 호출
print(x_train.shape)
torch.Size([1000, 9])
linear1 = torch.nn.Linear(9, 30, bias = True)
linear2 = torch.nn.Linear(30, 30, bias = True)
linear3 = torch.nn.Linear(30, 1, bias = True)
relu = torch.nn.ReLU()

# layer로 linear 함수 사용
# 활성화 함수 relu 함수 사용
model = torch.nn.Sequential(linear1, relu,
                           linear2, relu, 
                           linear3).cuda()
# layer 작동 순서
# 모델 학습

loss = torch.nn.MSELoss().cuda()
# 회귀 문제이기 때문에 loss 함수로 MSELoss를 사용

optimizer = optim.SGD(model.parameters(), lr = 0.001)
# optimizer로 SGD 사용

nb_epochs = 5000

for epoch in range(nb_epochs + 1):
    
    hypothesis = model(x_train)
    
    cost = loss(hypothesis, y_train.unsqueeze(1))
    
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    if epoch % 100 == 0:
        print(epoch, cost.item())
        
# SGD를 사용할때 처음 cost 값이 크게 나와 lr를 0.1에서 0.001까지 줄여가며 조절
# cost가 유의미하게 크게 줄어드는것을 고려하여 epoch를 1000에서 3000, 5000, 으로 늘림
0 19312.513671875
100 3104.072265625
200 2342.246826171875
300 1899.012939453125
400 2171.9755859375
500 1832.5203857421875
600 1659.43212890625
700 1576.8045654296875
800 1458.279052734375
900 1401.5426025390625
1000 1301.5474853515625
1100 1256.3345947265625
1200 1236.3128662109375
1300 1080.5673828125
1400 1088.3709716796875
1500 1021.0570678710938
1600 977.6937866210938
1700 972.03857421875
1800 903.642822265625
1900 839.2060546875
2000 871.184814453125
2100 769.9696655273438
2200 826.9214477539062
2300 725.1409301757812
2400 839.95751953125
2500 794.619873046875
2600 780.0992431640625
2700 639.16748046875
2800 703.5933837890625
2900 634.4940185546875
3000 617.7144775390625
3100 639.7200317382812
3200 680.9598388671875
3300 618.779541015625
3400 595.28955078125
3500 592.1597900390625
3600 616.5580444335938
3700 545.1934814453125
3800 523.733154296875
3900 555.3435668945312
4000 603.1419677734375
4100 564.524658203125
4200 478.7824401855469
4300 538.4795532226562
4400 464.1126403808594
4500 529.2308959960938
4600 411.4157409667969
4700 382.4579162597656
4800 469.8476867675781
4900 410.4792175292969
5000 436.9608154296875
# x_test로 모델 예측

with torch.no_grad():
    hypothesis = model(x_test)
    predict = hypothesis
    submit['count'] = predict.detach().cpu().numpy().astype(float)
    print(submit)
      id       count
0      0   13.256634
1      1  261.648926
2      2  142.918869
3      3   25.740717
4      4   74.538879
..   ...         ...
322  322   28.640741
323  323   32.878277
324  324   44.834766
325  325  192.338501
326  326  187.779282

[327 rows x 2 columns]
# 예측값 제출
submit.to_csv('submission.csv', index = False)
# 따릉이 사용자 예측문제를 DNN 방식으로 풀어보았습니다.
# 3개의 Linear layer와 relu함수를 활성화 함수로 사용하였고
# optimizer는 SGD를 사용하였습니다. 여기서 lr은 cost가 수렴하는 모습을 보일때 까지 0.1에서 0.001까지 조절하게 되었습니다.
# cost가 유의미하게 크게 줄어드는것을 고려하여 epoch를 1000에서 3000, 5000 으로 늘려서 모델을 학습시켰습니다.
# 회귀 문제이기 때문에 loss 함수로 MSELoss를 사용하였습니다.
# optimizer를 Adam으로 사용하고 dropout 기법 또한 사용한다면 큰 성능 향상이 있을것입니다.